In [1]:
import os
import json
import pandas as pd

In [2]:
from webapitools import get_entity_mentions
from nlptools import get_ner_tokens, get_ner, get_sentiment, is_hit
from datetime import datetime

In [3]:
entity_name = "uber"
size = 100

In [4]:
def clean_string(s):
    return ''.join([c if c.isalnum() else '_' for c in s])

In [5]:
outpath = 'data/search/comment/q={}'.format(clean_string(entity_name))
os.makedirs(outpath, exist_ok=True)

manifestpath = os.path.join(outpath, 'manifest.csv')
if not os.path.isfile(manifestpath):
    with open(manifestpath, 'w+') as f:
        f.write('query, created_utcnow_isoformat\n')
    manifestdf = None
else:
    manifestdf = pd.read_csv(manifestpath)

In [6]:
comments = get_entity_mentions(entity_name, size=size)
utciso = datetime.utcnow().isoformat()
entity_name_tokenized = get_ner_tokens(entity_name)

/Users/tianlechen/GitHub/astroturf/.venv/lib/python3.7/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [7]:
comments_hit = []
for comment in comments:
    ner = get_ner(comment['body'])
    comment_id = comment['id']
    if is_hit(ner, entity_name_tokenized):
        comment.update({'sentiment': get_sentiment(comment['body'])})
        comment_outfname = os.path.join(outpath, '{}.json'.format(comment_id))
        if not os.path.isfile(comment_outfname):
            with open(comment_outfname, 'w+') as f:
                json.dump(comments_hit, f, indent=4)

In [8]:
with open(manifestpath, 'a+') as f:
    f.write('{query}, {utciso}\n'.format(
        query=entity_name,
        utciso=utciso, 
    ))

In [ ]:
!gsutil -m rsync -r data/search/comment gs://astroturf-dev/search/comment


both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

